In [ ]:
import numpy as np
import torch

In [ ]:
def bin2tensor(filePath,datatype=np.int64):
    tensor = np.fromfile(filePath,dtype=datatype)
    return tensor
def loadingNode(filePath,rank):
    NodeFilePath=filePath+"part"+str(rank)+"/NodeIds.bin"
    NodeList = bin2tensor(NodeFilePath,datatype=np.int64)
    return NodeList
def loadingHalo(filePath,rank1,rank2):
    HaloFilePath=filePath+"part"+str(rank1)+"/halo_"+str(rank2)+".bin"
    HaloList = bin2tensor(HaloFilePath,datatype=np.int64)
    return HaloList

In [ ]:
GMap=torch.Tensor(200).to(torch.int64)
raw_path="/raid/bear/test_part_dataset/"
sum = 0
## 构建全局map
for rank in range(4):
    nodeids=loadingNode(raw_path,rank)
    nodeNUM = len(nodeids)
    newIndex = torch.arange(nodeNUM) + sum
    sum += nodeNUM
    GMap[nodeids]=newIndex


In [ ]:
HaloList = loadingHalo(raw_path,0,1)
HaloList = GMap[HaloList]
HaloList

In [27]:
def loadingFeat(FilePath,nodeNUM,FeatLen):
    featFilePath=FilePath+"raw/feats.bin"
    fpr = np.memmap(featFilePath, dtype='float32', mode='r', shape=(nodeNUM,FeatLen))
    return fpr

def fetchFeat(fpr,indices):
    feats = fpr[indices]
    return feats

In [30]:
fpr=loadingFeat(raw_path,200,100)
feats=fetchFeat(fpr,GMap[:50])
feats.tofile("test.bin")

In [ ]:
class DataAdapter:
    def __init__(self, FilePath, nodeNUM, FeatLen):
        # "/raid/bear/test_part_dataset/"
        self.raw_path = FilePath
        self.featFilePath = FilePath + "raw/feats.bin"
        self.nodeNUM = nodeNUM
        self.FeatLen = FeatLen

    def fetchFeat(self, indices):
        fpr = np.memmap(self.featFilePath, dtype='float32', mode='r', shape=(self.nodeNUM, self.FeatLen))
        feats = fpr[indices]
        return feats

    def fetchLabel(labelFilePath, indices):
        label_data = np.fromfile(labelFilePath, dtype=np.int64)
        label_selected_data = label_data[indices]
        return label_selected_data

    def bin2tensor(filePath, datatype=np.int64):
        tensor = np.fromfile(filePath, dtype=datatype)
        return tensor

    def loadingHalo(filePath, rank1, rank2):
        HaloFilePath = filePath + "part" + str(rank1) + "/halo_" + str(rank2) + ".bin"
        HaloList = self.bin2tensor(HaloFilePath, datatype=np.int64)
        return HaloList
    
    def loadingNode(filePath, rank):
        NodeFilePath = filePath + "part" + str(rank) + "/NodeIds.bin"
        NodeList = self.bin2tensor(NodeFilePath, datatype=np.int64)
        return NodeList